In [ ]:
# create spark session
import uuid
import time

from pyspark.sql import Row, DataFrame
from pyspark.sql.functions import explode, count,col, format_number
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, ArrayType

from app.common.json_util import *
from xinghe.spark import *
from xinghe.s3 import *

config = {
    "spark_conf_name": "spark_2",
    "skip_success_check": True,
    "spark.yarn.queue": "qa",
    # "spark.dynamicAllocation.maxExecutors":120,
    "spark.executor.memory": "80g",
    "spark.speculation": "true",     # 启用推测执行
    "maxRecordsPerFile": 200000,      # 增加每文件记录数以减少总文件数
    "output_compression": "gz",
}


spark = new_spark_session("cc_domain_hash", config)
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

In [ ]:
input_paths = [
    "s3://web-parse-huawei/CC/pre-dedup/v008/unique_html/CC-MAIN-2013-20/", # 第一批
]


input_df = read_any_path(spark, ",".join(input_paths), config)

In [ ]:
from pyspark.sql import Row, DataFrame, Window
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
import xxhash
from urllib.parse import urlparse
import json
from pyspark.sql.functions import monotonically_increasing_id, expr


# 配置参数
hash_count = 10000  # 域名哈希桶数量10000
target_records_per_file = 100000  # 每个文件的目标记录数100000
# 分区设置10000
numPartitions_id = 10000
numPartitions_subpath = 15000

# 定义提取domain的UDF
def extract_domain(url):
    if url is None:
        return None
    try:
        hostname = urlparse(url).hostname
        return hostname.lower() if hostname else None
    except Exception as e:
        return None

# 定义计算domain_hash_id的UDF
def compute_domain_hash(domain):
    if domain is None:
        return None
    return xxhash.xxh64_intdigest(domain) % hash_count

# 注册UDF以在DataFrame操作中使用
extract_domain_udf = F.udf(extract_domain, StringType())
compute_domain_hash_udf = F.udf(compute_domain_hash, IntegerType())


def process_with_df_api_and_file_indices(df, target_records_per_file):
    """
    一体化处理函数，完成以下步骤：
    1. 从JSON提取字段
    2. 添加domain和domain_hash_id
    3. 计算file_idx（确保为非负值）
    4. 更新sub_path，包含domain_hash_id和file_idx
    5. 重新构建JSON（防止字段重复）
    """
    
    # 1. 提取url用于计算域名和哈希
    extracted_df = df.withColumn("url", F.get_json_object(F.col("value"), "$.url")) \
        .withColumn("original_sub_path", F.get_json_object(F.col("value"), "$.sub_path"))
    
    # 2. 提取域名和计算哈希ID
    processed_df = extracted_df \
        .withColumn("domain", extract_domain_udf(F.col("url"))) \
        .withColumn("domain_hash_id", compute_domain_hash_udf(F.col("domain"))) \
        .filter(F.col("domain_hash_id").isNotNull())
    
    # 3. 使用高效方法计算file_idx，确保为非负值
    # 添加唯一ID
    df_with_id = processed_df.withColumn("_uid", F.abs(monotonically_increasing_id()))

    # 使用更细粒度的预分区
    df_with_id_repartitioned = df_with_id.repartition(numPartitions_id, "domain_hash_id")

    # 然后再应用窗口函数
    df_with_indices = df_with_id_repartitioned \
        .withColumn("rel_pos", F.row_number().over(Window.partitionBy("domain_hash_id").orderBy("_uid"))) \
        .withColumn("file_idx", ((F.col("rel_pos") - 1) / target_records_per_file).cast("long"))
    
    # 4. 创建新的sub_path字段，包含domain_hash_id和file_idx
    def update_complete_json(json_str, domain, domain_hash_id, file_idx, original_sub_path):
        """更新JSON，保留所有原始字段"""
        try:
            data = json_loads(json_str)
            # 添加新字段
            data["domain"] = domain
            data["domain_hash_id"] = domain_hash_id
            data["file_idx"] = file_idx
            # 更新sub_path
            data["sub_path"] = f"{domain_hash_id}/{original_sub_path if original_sub_path else ''}/{file_idx}"
            return json_dumps(data)
        except Exception as e:
            print(f"JSON更新错误: {e}")
            return json_str
    
    # 注册UDF
    update_json_udf = F.udf(update_complete_json, StringType())
    
    # 应用UDF
    final_df = df_with_indices.withColumn(
        "value",
        update_json_udf(
            F.col("value"),  # 原始完整JSON
            F.col("domain"),
            F.col("domain_hash_id"),
            F.col("file_idx"),
            F.col("original_sub_path")
        )
    ).withColumn(
        "sub_path",
        F.concat(
            F.col("domain_hash_id").cast("string"), 
            F.lit("/"), 
            F.col("original_sub_path"), 
            F.lit("/"),
            F.col("file_idx").cast("string")
        )
    ).select("value", "sub_path")  # 只保留value和sub_path列
    
    return final_df


In [ ]:
# ===== 主处理流程 =====

# 使用一体化处理函数处理数据
prepared_df = process_with_df_api_and_file_indices(input_df, target_records_per_file)

regrouped_df = prepared_df.repartition(numPartitions_subpath, "sub_path")

# 验证分区结果是否符合预期
# quick_validation(regrouped_df)

# # 获取分区数量
# num_partitions = regrouped_df.rdd.getNumPartitions()
# print(f"当前DataFrame有 {num_partitions} 个分区")

In [ ]:
# 写入中间数据
# output_path = "s3://qa-huawei/chupei/cc-domain-centric-store/data-mid-1dump-0412-v2/"
output_path = "s3://web-parse-hw60p/CC-domain/"
write_any_path(regrouped_df, output_path, config)
